In [1]:
import os
import librosa
import math
import json
DATASET_PATH = "/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original"
JSON_PATH="/kaggle/working/alldata.json"
SAMPLE_RATE =22050
DURATION = 30
SAMPLES_PER_TRACK = SAMPLE_RATE*DURATION

**Preparing Data**

In [2]:
def save_mfcc(dataset_path,jsonPath,n_mfcc=13,n_fft=2048,hop_length = 25,num_segments=5):
    #dect to store data
    data ={
        "mapping":[],
        "mfcc":[],
        "labels":[]
    }
    num_seamples_per_segment = int(SAMPLES_PER_TRACK/num_segments)
    num_mfcc_vector_per_segment = math.ceil(num_seamples_per_segment/hop_length)
    #loop through genra
    for i,(dirpath,dirnames,filenames) in enumerate(os.walk(dataset_path)):
        if dirpath is not dataset_path:
            #save labels
            dirpath_components = dirpath.split("/")
            semantic_label = dirpath_components[-1]
            data["mapping"].append(semantic_label)
            print("\nProcessing {}".format(semantic_label))
            for f in filenames:
                try:
                    #load file path
                    file_path = os.path.join(dirpath,f)
                    signal , sr = librosa.load(file_path,sr=SAMPLE_RATE)
                    #process segemnts and extracting mfcc and storing data
                    for s in range(num_segments):
                        start_sample = num_seamples_per_segment * s
                        finish_sample= start_sample + num_seamples_per_segment
                        #store mfcc for segment if it has the expected length
                        mfcc = librosa.feature.mfcc(y=signal[start_sample:finish_sample],
                                                sr=sr,
                                                n_mfcc=n_mfcc,
                                                n_fft=n_fft,
                                                hop_length=hop_length)
                        mfcc = mfcc.T
                        data["mfcc"].append(mfcc.tolist())
                        print("mfcc addded")
                        # "i" is the number of genra 
                        data["labels"].append(i-1)
                        print("\n{},segment:{}".format(file_path,s))
                except:
                      print("An exception occurred")
    with open(JSON_PATH,"w") as fp:
        json.dump(data,fp,indent=4)

In [3]:
#Done , Commented as the output was made 
#save_mfcc(DATASET_PATH,JSON_PATH,num_segments=10)